In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [2]:

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression, Lasso
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from tabulate import tabulate


from google.colab import drive
drive.mount('/content/drive')



Mounted at /content/drive


In [4]:
attributes = pd.read_csv('/content/drive/MyDrive/airline_data/df_final_cleaned.csv',low_memory=False)
attributes

,SourceAirport,DestinationAirport,dep,capacity,days_left_result,cre,mean_confirmed_fare,cumulative number of seats booked,cumulative number of bookings,number_of_bookings_on_given_day,...,departure_day,departure_month,DepartureDate,day_type_weekend,departure_hour,percentage_diff,Conversion Rate,departure_time,time_category,day_of_week
0,SKG,ATH,2023-10-07 07:00:00,186.0,86,2023-07-13,55.0,2.0,2.0,1.0,...,7,October,2023-10-07,1,7,1.528000,0.055794,07:00:00,Morning,Saturday
1,HER,ATH,2023-08-10 07:00:00,186.0,84,2023-05-17,56.5,11.0,4.0,2.0,...,10,August,2023-08-10,0,7,0.391327,0.199650,07:00:00,Morning,Thursday
2,HER,ATH,2023-10-16 07:00:00,186.0,67,2023-08-09,15.0,4.0,2.0,1.0,...,16,October,2023-10-16,0,7,19.172000,0.210000,07:00:00,Morning,Monday
3,HER,ATH,2023-10-16 07:00:00,186.0,68,2023-08-08,15.0,2.0,1.0,1.0,...,16,October,2023-10-16,0,7,2.926000,0.210000,07:00:00,Morning,Monday
4,HER,ATH,2023-08-10 07:00:00,186.0,72,2023-05-30,49.5,21.0,9.0,2.0,...,10,August,2023-08-10,0,7,0.529293,0.199650,07:00:00,Morning,Thursday
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33973,ATH,HER,2023-09-22 23:30:00,186.0,76,2023-07-08,24.0,2.0,1.0,1.0,...,22,September,2023-09-22,0,23,2.252083,0.170659,23:30:00,Late Night,Friday
33974,ATH,HER,2023-09-22 23:30:00,186.0,39,2023-08-14,28.0,14.0,8.0,1.0,...,22,September,2023-09-22,0,23,2.319643,0.170659,23:30:00,Late Night,Friday
33975,ATH,HER,2023-09-22 23:30:00,186.0,44,2023-08-09,28.0,13.0,7.0,1.0,...,22,September,2023-09-22,0,23,2.292857,0.170659,23:30:00,Late Night,Friday
33976,ATH,HER,2023-09-22 23:30:00,186.0,54,2023-07-30,24.0,10.0,6.0,1.0,...,22,September,2023-09-22,0,23,2.551667,0.170659,23:30:00,Late Night,Friday


In [5]:
attributes.columns

Index(['SourceAirport', 'DestinationAirport', 'dep', 'capacity',
       'days_left_result', 'cre', 'mean_confirmed_fare',
       'cumulative number of seats booked', 'cumulative number of bookings',
       'number_of_bookings_on_given_day', 'number_of_seatsbooked_on_given_day',
       'load factor', 'baseline', 'Departure_Time', 'departure_day',
       'departure_month', 'DepartureDate', 'day_type_weekend',
       'departure_hour', 'percentage_diff', 'Conversion Rate',
       'departure_time', 'time_category', 'day_of_week'],
      dtype='object')

In [53]:
def get_per_day_trajectories(bookings):
    '''
    Given the cleaned bookings and searches data, obtain the state, action, and reward trajectories per day.

    Output should be a list of trajectories corresponding to booking decisions that happen on a certain flight.
    A trajectory is a list of booking events that happen for a flight, so the number of trajectories == no of flights.

    A trajectory is given by [[s1,a1,r1], [s2,a2,r2], [s3,a3,r3], ....],
    where si's are the states, ai's are the actions, and ri's are the rewards recieved.

    s = [no_of_customers_arriving_on_day, no_of_seats_booked_before_day, days_left, route_information, date_time_info]
    a = (avg) price charged on that day
    r = total money made on that day

    All data should be float or integer valued

    '''

    unique_combinations_flight = bookings[['SourceAirport', 'DestinationAirport', 'dep']].drop_duplicates()
    unique_flights = list(unique_combinations_flight.itertuples(index=False, name=None))

    # Step 2: Create Empty Trajectories
    trajectories = {flight: [] for flight in unique_flights}
    for flight in unique_flights:
        flight_data = bookings[(bookings['SourceAirport'] == flight[0]) &
                                (bookings['DestinationAirport'] == flight[1]) &
                                (bookings['dep'] == flight[2])]
        flight_data_sorted = flight_data.sort_values(by='days_left_result', ascending=False)

        flight_info_list = []

        for index, row in flight_data_sorted.iterrows():
            s = []
            a = []
            r = []
            # current 'days_left'
            current_days_left = row['days_left_result']

            # find 'days_left' with closet larger value
            previous_rows = flight_data_sorted[flight_data_sorted['days_left_result'] > current_days_left]
            if not previous_rows.empty:
                no_of_seats_booked_before_day = previous_rows.iloc[-1]['cumulative number of seats booked']
            else:
                no_of_seats_booked_before_day = 0

            no_of_customers_arriving_on_day = row['cumulative number of seats booked']
            days_left = row['days_left_result']
            route_information = (row['SourceAirport'], row['DestinationAirport'])
            date_time_info = row['dep']
            avg_price_charged_on_that_day = row['mean_confirmed_fare']
            s.append([no_of_customers_arriving_on_day, no_of_seats_booked_before_day,
                                    days_left,
                                    route_information,
                                    date_time_info])
            a.append(avg_price_charged_on_that_day)
            r.append(avg_price_charged_on_that_day*no_of_customers_arriving_on_day)
            flight_info_certain_day = [s,a,r]
            flight_info_list.append(flight_info_certain_day)


        trajectories[flight] = flight_info_list
    return trajectories


In [54]:
trajectories = get_per_day_trajectories(attributes)
for flight, trajectory in trajectories.items():
    print(f"flight: {flight}")
    for step in trajectory:
        s, a, r = step
        print(f"  state (s): {s}")
        print(f"  action (a): {a}")
        print(f"  reward (r): {r}")
    print("\n")



流式输出内容被截断，只能显示最后 5000 行内容。
  state (s): [[99.0, 93.0, 39, ('HER', 'ATH'), '2023-08-19 23:10:00']]
  action (a): [53.0]
  reward (r): [5247.0]
  state (s): [[101.0, 99.0, 38, ('HER', 'ATH'), '2023-08-19 23:10:00']]
  action (a): [47.235]
  reward (r): [4770.735]
  state (s): [[106.0, 101.0, 37, ('HER', 'ATH'), '2023-08-19 23:10:00']]
  action (a): [55.694]
  reward (r): [5903.564]
  state (s): [[106.0, 101.0, 37, ('HER', 'ATH'), '2023-08-19 23:10:00']]
  action (a): [55.694]
  reward (r): [5903.564]
  state (s): [[110.0, 106.0, 36, ('HER', 'ATH'), '2023-08-19 23:10:00']]
  action (a): [60.75]
  reward (r): [6682.5]
  state (s): [[111.0, 110.0, 35, ('HER', 'ATH'), '2023-08-19 23:10:00']]
  action (a): [59.0]
  reward (r): [6549.0]
  state (s): [[119.0, 111.0, 34, ('HER', 'ATH'), '2023-08-19 23:10:00']]
  action (a): [60.75]
  reward (r): [7229.25]
  state (s): [[120.0, 119.0, 33, ('HER', 'ATH'), '2023-08-19 23:10:00']]
  action (a): [59.0]
  reward (r): [7080.0]
  state (s): [[124.0, 120

In [ ]:
def get_per_booking_trajectories(bookings):
    '''
    Given the cleaned data, obtain the state, action, and reward for the booking
    Not neccesary
    '''

In [ ]:
class Policy:
    def __init__(S,A,P):
        for s in S:
            for t in S:
                return

## **Offline Evaluation and RL via Importance Sampling**

In [ ]:
def importance_sampling_estimator(trajectories, behaviour_policy,
                                  eval_policy, gamma):

    '''
    Given the data of the trajectories, derive an estimator for the discounted value
    of the eval_policy
    '''

    J = 0
    n = len(trajectories)
    for i in range(n):
        w = 1
        total_return = 0
        for t in range(len(trajectories[i])):
            w = w * eval_policy.evaluate(trajectories[i][t][1],trajectories[i][t][0]) / (
                behaviour_policy.evaluate(trajectories[i][t][1],trajectories[i][t][0]))

            total_return += (gamma**t) * trajectories[i][t][2]
        J += w * total_return
    return J/n

In [ ]:
def per_decision_importance_sampling(trajectories, behaviour_policy,
                                     eval_policy, gamma):
    '''
    Given the data of the trajectories, derive an estimator for the discounted value
    of the eval_policy with the per decision estimator
    '''

    J = 0
    n = len(trajectories)
    for i in range(n):
        w = 1
        for t in range(len(trajectories[i])):
            w = w * eval_policy.evaluate(trajectories[i][t][1],trajectories[i][t][0]) / (
                behaviour_policy.evaluate(trajectories[i][t][1],trajectories[i][t][0]))

            J += w * (gamma**t) * trajectories[i][t][2]
    return J/n

In [ ]:
def off_policy_gradient(trajectories, behaviour_policy,
                        eval_policy, gamma):



In [ ]:
#main
